In [1]:
import json
import math

# Charger le fichier JSON
with open('avito_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Fonction pour supprimer les clés avec des valeurs NaN
def remove_nan_keys(obj):
    if isinstance(obj, dict):
        return {k: remove_nan_keys(v) for k, v in obj.items() 
                if not (isinstance(v, float) and math.isnan(v))}
    elif isinstance(obj, list):
        return [remove_nan_keys(item) for item in obj]
    else:
        return obj

# Appliquer la suppression
data_clean = remove_nan_keys(data)

# Sauvegarder le résultat
with open('avito_data.json', 'w', encoding='utf-8') as f:
    json.dump(data_clean, f, indent=4, ensure_ascii=False)

In [7]:
import requests
import pandas as pd
import json
import time
from datetime import datetime
from pathlib import Path

# ============================================================================
# CONFIGURATION GRAPHQL
# ============================================================================

GRAPHQL_API_URL = "https://gateway.avito.ma/graphql"
GRAPHQL_API_HEADERS = {
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "Origin": "https://www.avito.ma",
    "Referer": "https://www.avito.ma/",
}

# REQUÊTE GRAPHQL POUR RÉCUPÉRER L'URL INDIVIDUELLE
GRAPHQL_URL_QUERY = """
query getAdDetails($adId: ID!) {
  getAd(adId: $adId) {
    ... on PublishedAd {
      adId
      title
      url
      category {
        name
        parent {
          name
        }
      }
      price {
        withCurrency
      }
    }
  }
}
"""

# ============================================================================
# FONCTIONS CORRIGÉES
# ============================================================================

def fetch_single_url_via_graphql(ad_id, retry=3, delay=1):
    """Récupère l'URL réelle pour une seule annonce via GraphQL"""
    variables = {"adId": str(ad_id)}
    
    for attempt in range(retry):
        try:
            response = requests.post(
                GRAPHQL_API_URL,
                headers=GRAPHQL_API_HEADERS,
                json={"query": GRAPHQL_URL_QUERY, "variables": variables},
                timeout=30
            )

            if response.status_code == 200:
                data = response.json()
                
                # Debug: afficher la réponse pour les premiers IDs
                if ad_id in [75962308, 75687396, 75446550]:  # Vos premiers IDs
                    print(f"      🔍 DEBUG Réponse pour {ad_id}: {json.dumps(data, ensure_ascii=False)[:300]}")
                
                if data.get("data") and data["data"].get("getAd"):
                    ad_data = data["data"]["getAd"]
                    url = ad_data.get("url")
                    if url:
                        return url
                    else:
                        print(f"      ⚠️  Pas d'URL dans la réponse pour {ad_id}")
                elif data.get("errors"):
                    error_msg = data['errors'][0].get('message', 'Unknown error')
                    print(f"      ❌ Erreur API pour {ad_id}: {error_msg}")
                    
                    # Si l'erreur indique que l'annonce n'existe pas
                    if "not found" in error_msg.lower() or "does not exist" in error_msg.lower():
                        return "NOT_FOUND"
                
            elif response.status_code == 429:  # Too Many Requests
                print(f"      ⚠️  Rate limit (429), pause de 30s...")
                time.sleep(30)
                continue
            elif response.status_code == 422:  # Unprocessable Entity
                print(f"      ⚠️  Erreur 422 pour {ad_id}")
                return None
            else:
                print(f"      ❌ Statut {response.status_code} pour {ad_id}")
                
        except requests.exceptions.RequestException as e:
            print(f"      ⚠️  Erreur réseau (tentative {attempt + 1}/{retry}): {e}")
        
        if attempt < retry - 1:
            time.sleep(delay)
    
    return None

def scrape_urls_individual(existing_data, batch_size=10, delay_between_batches=2):
    """
    Scrape les URLs réelles une par une via l'API GraphQL
    """
    if isinstance(existing_data, list):
        df = pd.DataFrame(existing_data)
    else:
        df = existing_data.copy()
    
    print(f"🎯 DÉBUT DU SCRAPING DES URLs (MODE INDIVIDUEL)")
    print(f"📊 Total d'annonces: {len(df)}")
    print(f"📦 Taille des lots: {batch_size}")
    print("=" * 70)
    
    urls_found = 0
    not_found_count = 0
    error_count = 0
    total_ads = len(df)
    
    # Ajouter les colonnes pour les résultats
    df['scraped_url'] = None
    df['url_status'] = 'pending'
    
    # Traiter par lots
    for i in range(0, total_ads, batch_size):
        batch_end = min(i + batch_size, total_ads)
        current_batch_size = batch_end - i
        batch_num = i // batch_size + 1
        total_batches = (total_ads + batch_size - 1) // batch_size
        
        print(f"\n🔄 Lot {batch_num}/{total_batches} (annonces {i+1}-{batch_end})")
        
        batch_success = 0
        batch_errors = 0
        
        for j in range(i, batch_end):
            ad_id = df.at[j, 'adId']
            current_num = j + 1
            
            print(f"      🔗 [{current_num}/{total_ads}] Récupération URL pour {ad_id}...", end=" ")
            
            url = fetch_single_url_via_graphql(ad_id)
            
            if url:
                if url == "NOT_FOUND":
                    df.at[j, 'scraped_url'] = None
                    df.at[j, 'url_status'] = 'not_found'
                    not_found_count += 1
                    print("❌ Annonce non trouvée")
                else:
                    df.at[j, 'scraped_url'] = url
                    df.at[j, 'url_status'] = 'found'
                    urls_found += 1
                    batch_success += 1
                    print("✅ Trouvée")
            else:
                df.at[j, 'scraped_url'] = None
                df.at[j, 'url_status'] = 'error'
                error_count += 1
                batch_errors += 1
                print("❌ Erreur")
            
            # Petite pause entre les requêtes
            if j < batch_end - 1:
                time.sleep(0.5)
        
        print(f"      📊 Résultats lot: {batch_success}✅ {batch_errors}❌")
        print(f"      📈 Progression globale: {urls_found}✅ {not_found_count}🔍 {error_count}❌")
        
        # Sauvegarde temporaire tous les 10 lots
        if batch_num % 10 == 0:
            timestamp = datetime.now().strftime('%H%M%S')
            temp_file = f"temp_urls_progress_{timestamp}.json"
            df.to_json(temp_file, orient='records', indent=2, force_ascii=False)
            print(f"      💾 Sauvegarde temporaire: {temp_file}")
        
        # Pause entre les lots
        if batch_end < total_ads:
            print(f"      ⏸️  Pause de {delay_between_batches} secondes...")
            time.sleep(delay_between_batches)
    
    print(f"\n{'=' * 70}")
    print(f"✨ SCRAPING TERMINÉ!")
    print(f"📊 RÉSULTATS FINAUX:")
    print(f"   ✅ URLs trouvées: {urls_found}/{total_ads}")
    print(f"   🔍 Annonces non trouvées: {not_found_count}/{total_ads}")
    print(f"   ❌ Erreurs: {error_count}/{total_ads}")
    print(f"   📈 Taux de succès: {(urls_found/total_ads)*100:.1f}%")
    
    return df

# ============================================================================
# SCRIPT PRINCIPAL CORRIGÉ
# ============================================================================

def main():
    print("╔═══════════════════════════════════════════════════════════════════╗")
    print("║          SCRAPER URLs RÉELLES AVITO - MODE INDIVIDUEL            ║")
    print("╚═══════════════════════════════════════════════════════════════════╝")
    
    # Charger vos données
    print("\n📂 Chargement de vos données...")
    try:
        with open('avito_data.json', 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
        print(f"✅ Données chargées: {len(existing_data)} annonces")
    except Exception as e:
        print(f"❌ Erreur chargement: {e}")
        return
    
    # Aperçu
    print(f"\n👀 Aperçu des données:")
    for i, ad in enumerate(existing_data[:3]):
        print(f"  {i+1}. ID: {ad['adId']} - {ad['title']}")
    
    # Configuration
    print(f"\n⚙️  Configuration du scraping:")
    
    # Tester d'abord avec un petit échantillon
    test_sample = input("Tester d'abord avec 3 annonces? (o/n) [o]: ").strip().lower() != 'n'
    
    if test_sample:
        sample_data = existing_data[:3]
        print("🧪 MODE TEST: 3 premières annonces seulement")
    else:
        use_sample = input("Utiliser un échantillon? (o/n) [n]: ").strip().lower() == 'o'
        if use_sample:
            sample_size = input(f"Taille échantillon (défaut: 100): ").strip()
            sample_size = int(sample_size) if sample_size else 100
            sample_data = existing_data[:sample_size]
        else:
            sample_data = existing_data
    
    batch_size = input("Taille des lots (défaut: 10): ").strip()
    batch_size = int(batch_size) if batch_size else 10
    
    delay = input("Délai entre les lots en secondes (défaut: 2): ").strip()
    delay = int(delay) if delay else 2
    
    print(f"\n🎯 CONFIGURATION FINALE:")
    print(f"📊 Annonces à traiter: {len(sample_data)}")
    print(f"📦 Taille des lots: {batch_size}")
    print(f"⏱️  Délai entre lots: {delay} secondes")
    
    confirmation = input("\nDémarrer le scraping? (o/n): ").strip().lower()
    if confirmation != 'o':
        print("❌ Annulé")
        return
    
    # Lancer le scraping
    print(f"\n🚀 DÉMARRAGE DU SCRAPING...")
    print("=" * 70)
    
    start_time = time.time()
    
    results_df = scrape_urls_individual(
        sample_data, 
        batch_size=batch_size, 
        delay_between_batches=delay
    )
    
    end_time = time.time()
    duration = end_time - start_time
    
    # Sauvegarder les résultats
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_dir = Path("avito_urls_results")
    output_dir.mkdir(exist_ok=True)
    
    # Fichier CSV
    csv_path = output_dir / f"avito_with_real_urls_{timestamp}.csv"
    results_df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    
    # Fichier JSON
    json_path = output_dir / f"avito_with_real_urls_{timestamp}.json"
    results_df.to_json(json_path, orient='records', indent=2, force_ascii=False)
    
    print(f"\n💾 Résultats sauvegardés:")
    print(f"   📄 CSV: {csv_path}")
    print(f"   📄 JSON: {json_path}")
    
    # Afficher les statistiques finales
    urls_found = len(results_df[results_df['url_status'] == 'found'])
    total_processed = len(results_df)
    
    print(f"\n{'=' * 70}")
    print("📊 RAPPORT FINAL")
    print(f"⏱️  Temps total: {duration:.2f} secondes")
    print(f"📊 Annonces traitées: {total_processed}")
    print(f"✅ URLs réelles trouvées: {urls_found}")
    
    if urls_found > 0:
        print(f"📈 Taux de succès: {(urls_found/total_processed)*100:.1f}%")
        print(f"\n👀 EXEMPLES D'URLs RÉELLES TROUVÉES:")
        success_data = results_df[results_df['url_status'] == 'found'].head(3)
        for _, row in success_data.iterrows():
            print(f"  🔗 {row['adId']}: {row['scraped_url']}")
    else:
        print(f"\n❌ Aucune URL trouvée. Problèmes possibles:")
        print("   - L'API a changé")
        print("   - Les annonces ont expiré")
        print("   - Restrictions d'accès")

if __name__ == "__main__":
    main()

╔═══════════════════════════════════════════════════════════════════╗
║          SCRAPER URLs RÉELLES AVITO - MODE INDIVIDUEL            ║
╚═══════════════════════════════════════════════════════════════════╝

📂 Chargement de vos données...
✅ Données chargées: 21719 annonces

👀 Aperçu des données:
  1. ID: 75962308 - iphone xr bon etat 
  2. ID: 75687396 - iphone 7plus 
  3. ID: 75446550 - Nokia x100 6g 128g

⚙️  Configuration du scraping:


Tester d'abord avec 3 annonces? (o/n) [o]:  o


🧪 MODE TEST: 3 premières annonces seulement


Taille des lots (défaut: 10):  15
Délai entre les lots en secondes (défaut: 2):  2



🎯 CONFIGURATION FINALE:
📊 Annonces à traiter: 3
📦 Taille des lots: 15
⏱️  Délai entre lots: 2 secondes



Démarrer le scraping? (o/n):  o



🚀 DÉMARRAGE DU SCRAPING...
🎯 DÉBUT DU SCRAPING DES URLs (MODE INDIVIDUEL)
📊 Total d'annonces: 3
📦 Taille des lots: 15

🔄 Lot 1/1 (annonces 1-3)
      ⚠️  Erreur 422 pour 75962308ur 75962308... 
❌ Erreur
      🔗 [2/3] Récupération URL pour 75687396...       ⚠️  Erreur 422 pour 75687396
❌ Erreur
      🔗 [3/3] Récupération URL pour 75446550...       ⚠️  Erreur 422 pour 75446550
❌ Erreur
      📊 Résultats lot: 0✅ 3❌
      📈 Progression globale: 0✅ 0🔍 3❌

✨ SCRAPING TERMINÉ!
📊 RÉSULTATS FINAUX:
   ✅ URLs trouvées: 0/3
   🔍 Annonces non trouvées: 0/3
   ❌ Erreurs: 3/3
   📈 Taux de succès: 0.0%

💾 Résultats sauvegardés:
   📄 CSV: avito_urls_results\avito_with_real_urls_20251130_015821.csv
   📄 JSON: avito_urls_results\avito_with_real_urls_20251130_015821.json

📊 RAPPORT FINAL
⏱️  Temps total: 2.08 secondes
📊 Annonces traitées: 3
✅ URLs réelles trouvées: 0

❌ Aucune URL trouvée. Problèmes possibles:
   - L'API a changé
   - Les annonces ont expiré
   - Restrictions d'accès
